In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-dataset/sentiment_analysis.csv


In [2]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

In [3]:
from transformers import BertTokenizer, BertModel
import torch

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Move the model to the GPU
model.to(device)
device

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

device(type='cuda')

In [4]:
df = pd.read_csv("/kaggle/input/sentiment-analysis-dataset/sentiment_analysis.csv")

In [5]:
df.head(5)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [6]:
df.drop(['id'], axis = 1, inplace = True)

In [7]:
target = df["label"]
data = df.drop(['label'], axis = 1)

remove puncation

In [8]:
data['tweet'] = data['tweet'].str.replace(r'[^\w\s]+', '')

remove urls

In [9]:
import re
def remove_urls(text, replacement_text=""):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text_without_urls = url_pattern.sub(replacement_text, text)
    return text_without_urls
data['tweet'] = data['tweet'].apply(remove_urls)

In [10]:
data.head(5)

,tweet
0,#fingerprint #Pregnancy Test #android #apps #...
1,Finally a transparant silicon case ^^ Thanks t...
2,We love this! Would you go? #talk #makememorie...
3,I'm wired I know I'm George I was made that wa...
4,What amazing service! Apple won't even talk to...


remove stop words using spacy

In [11]:
import spacy
stop_words = spacy.load("en_core_web_sm")
def stopword_remove(text):
    doc = stop_words(text)
    filtered_words = [token.text for token in doc if not token.is_stop]
    clean_text = ' '.join(filtered_words)
    return clean_text
data['tweet'] = data['tweet'].apply(stopword_remove)

In [12]:
data.head(5)

,tweet
0,# fingerprint # Pregnancy Test # android # a...
1,Finally transparant silicon case ^^ Thanks unc...
2,love ! ? # talk # makememories # unplug # rela...
3,wired know George way ;) # iphone # cute # dav...
4,amazing service ! Apple wo talk question pay $...


tokenize the data

In [13]:
tokenizer = TweetTokenizer()
data['tweet'] = data['tweet'].apply(tokenizer.tokenize)

lowercasing

In [14]:
def lowercasing(text):
    n = len(text)
    for i in range(n):
        text[i] = text[i].lower()
    return text
data['tweet'] = data['tweet'].apply(lowercasing)

Stemming

In [15]:
porter_stemmer = PorterStemmer()
def stemmer(words):
    stemmed_words = [porter_stemmer.stem(word) for word in words]
    return stemmed_words
data['tweet'] = data['tweet'].apply(stemmer)

lemmatization

In [16]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [17]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
lemmatizer = WordNetLemmatizer()
def lemma(words):
    pos_tags = pos_tag(words)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]
    return lemmatized_words
data['tweet'] = data['tweet'].apply(lemma)

In [18]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=data['tweet'], vector_size=100, window=5, min_count=1, workers=4)

In [19]:
def get_word2vec_embeddings(tokens):
    embeddings = [model.wv[token] for token in tokens if token in model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(100)

data['tweet'] = data['tweet'].apply(get_word2vec_embeddings)

In [20]:
X = np.vstack(data['tweet'].values)

In [21]:
X.shape

(7920, 100)

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

# Initialize and train the GaussianNB model
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Make predictions
y_pred = gnb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8491161616161617
